In [1]:
import requests
from urllib.parse import quote
import base64
import time
from datetime import datetime

In [2]:
class TippiLink:
    def __init__(self, username, password, host="192.168.0.1"):
        self._username = username
        self._password = password
        self._host = host

    def _get_headers(self):
        headers = {
            'Referer': 'http://%s/userRpm/MenuRpm.htm' % self._host,
            'Authorization': self._auth_header(),
        }
        return headers

    def _auth_header(self):
        auth_bytes = bytes(self._username+':'+self._password, 'utf-8')
        auth_b64_bytes = base64.b64encode(auth_bytes)
        auth_b64_str = str(auth_b64_bytes, 'utf-8')

        auth_str = quote('Basic {}'.format(auth_b64_str))
        return auth_str
    
    def _get_mac_from_page(self, pageName):
        
        try:

            response = requests.get('http://%s/userRpm/%s.htm' % (self._host, pageName), headers=self._get_headers())
            content = response.content

            ini = 'hostList = new Array('
            fin = ');'
            
            # nuevo cod
            texto = str(content)[str(content).index(ini)+len(ini)+2:].replace('\\n', '').replace(', 0,0 ', '')
            texto = texto[:texto.index(fin)].replace('-', ':').lower()

            lista = [l.replace("\"", "").replace(" ", "") for l in texto.split(',')]
            nueva_lista = [lista[i:i+4] for i in range(0, len(lista), 4) if lista[i+1] in ['0', '1', '8']]

            return nueva_lista
        
        except:
            return "Error"
    
    def get_all_macs(self):
        page = '1'
        macs = []
        mac_stat = []

        while True:
            arr_in = self._get_mac_from_page("WlanStationRpm.htm?Page="+page)
            arr = [l[0] for l in arr_in]
            page = str(int(page) + 1)

            if set(arr).issubset(set(macs)):
                break
            else:
                macs.extend(arr)
                mac_stat.extend(arr_in)

        mac_stat.sort(key=lambda x:int(x[2]), reverse=True)
        macs = [m[0] for m in mac_stat]
        return mac_stat #macs #
    
    def estadisticas(self):
        now = datetime.now()
        timestamp = datetime.timestamp(now)
        ini = 'wlanHostPara = new Array('
        ini1 = 'statistList = new Array('
        fin = ');'

        try:
            # numero de conectados
            nconx = len(self.get_all_macs())
            
            # enviados y recibidos
            response = requests.get('http://%s/userRpm/%s.htm' % (self._host, 'StatusRpm.htm'), headers=tl._get_headers())
            content = response.content
            texto = str(content)[str(content).index(ini1)+len(ini1)+2:]
            statistList = texto[:texto.index(fin)].split(',')

            b_rec = statistList[0]
            b_env = statistList[1]

            return [timestamp, nconx, b_rec, b_env]
        except:
            return "Error"
        
    def restart_router(self):
        params = (('Reboot', 'Reboot'),)
        try:
            res = requests.get('http://%s/userRpm/SysRebootRpm.htm' % self._host,
                                headers=self._get_headers(), params=params)
        except:
            pass
    

In [3]:
# https://pypi.org/project/python-dotenv/
from dotenv import load_dotenv
import os

load_dotenv()
tplink_id = os.getenv("TPLINK_ID")
tplink_pw = os.getenv("TPLINK_SECRET")

tl = TippiLink(tplink_id, tplink_pw, "192.168.66.18")


In [4]:
print(tl.estadisticas())

[1613226917.920898, 5, '0', ' 1893530']


In [10]:
tl.get_all_macs()

[['00:19:e0:a5:1f:20', '8', '1951193', '2395050'],
 ['78:40:e4:9c:4d:3b', '1', '804680', '828949'],
 ['b8:ae:ed:5e:0d:eb', '1', '342876', '363766'],
 ['b8:ae:ed:e7:a3:3e', '1', '130027', '113028'],
 ['40:72:02:39:03:3d', '1', '53557', '42556'],
 ['38:80:df:b4:fd:dd', '1', '50039', '52131'],
 ['00:11:0a:12:26:d3', '1', '18379', '13494'],
 ['b8:8a:60:2d:70:94', '1', '5796', '6109'],
 ['ec:1d:7f:80:bd:89', '1', '3775', '2367'],
 ['0c:ec:84:7c:8b:29', '1', '796', '616'],
 ['60:af:6d:6f:d8:49', '1', '4', '6']]

Current Status - the connected wireless station's running status, one of 
STA-AUTH 0 
STA-ASSOC 1 
AP-UP 8 

STA-JOINED / 
WPA / 
WPA-PSK / 
WPA2 / 
WPA2-PSK / 

AP-DOWN


10 ?